In [2]:
import numpy as np
import pickle

# Load normalized signals
X_all_normalized = np.load('preprocessed_data/X_all_normalized.npy')

# Load beat symbols
with open('preprocessed_data/sym_all.pkl', 'rb') as f:
    sym_all = pickle.load(f)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import time

# Assume `beat_types` is the list of your 14 target class symbols
beat_types = sorted(list(set(sym_all)))  # or define manually if needed
label_map = {sym: i for i, sym in enumerate(beat_types)}
y_labels = np.array([label_map[s] for s in sym_all])
y_onehot = to_categorical(y_labels, num_classes=len(beat_types))

# Reshape input for DNN (no channels, just flat vectors)
X = X_all_normalized
y = y_onehot

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y_labels, test_size=0.2, random_state=42)

# Define DNN model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(len(beat_types), activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
start_time = time.time()
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))
end_time = time.time()

# Total training time
print(f"Total training time: {end_time - start_time:.2f} seconds")

2025-05-07 09:55:49.573905: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-05-07 09:55:49.573946: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-07 09:55:49.573970: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-07 09:55:49.574109: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-07 09:55:49.574127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/15


2025-05-07 09:55:51.704446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1364/1364 ━━━━━━━━━━━━━━━━━━━━ 29s 20ms/step - accuracy: 0.4693 - loss: 5.6045 - val_accuracy: 0.5678 - val_loss: 1.3211
Epoch 2/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 26s 19ms/step - accuracy: 0.7070 - loss: 1.1005 - val_accuracy: 0.7443 - val_loss: 0.8907
Epoch 3/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 26s 19ms/step - accuracy: 0.7147 - loss: 1.0195 - val_accuracy: 0.7539 - val_loss: 0.7936
Epoch 4/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 26s 19ms/step - accuracy: 0.7237 - loss: 1.0046 - val_accuracy: 0.7387 - val_loss: 0.8533
Epoch 5/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 26s 19ms/step - accuracy: 0.7291 - loss: 1.0068 - val_accuracy: 0.7498 - val_loss: 0.8165
Epoch 6/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 26s 19ms/step - accuracy: 0.7264 - loss: 1.0426 - val_accuracy: 0.7629 - val_loss: 0.8445
Epoch 7/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 27s 19ms/step - accuracy: 0.7294 - loss: 1.0466 - val_accuracy: 0.7794 - val_loss: 0.7640
Epoch 8/15
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 27s 19ms/step - accuracy: 0.7268 - loss: 1.08

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │     1,106,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │         1,806 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,823,532 (14.59 MB)

 Trainable params: 1,273,998 (4.86 MB)

 Non-trainable params: 1,536 (6.00 KB)

 Optimizer params: 2,547,998 (9.72 MB)